In [1]:
%matplotlib
%matplotlib inline
import numpy
import matplotlib.pyplot as plt

Using matplotlib backend: TkAgg


In [2]:
import gillespy2
class MichaelisMenten(gillespy2.Model):
     def __init__(self, parameter_values=None):
            #initialize Model
            gillespy2.Model.__init__(self, name="Michaelis_Menten")
            
            #parameters
            rate1 = gillespy2.Parameter(name='rate1', expression= 0.0017)
            rate2 = gillespy2.Parameter(name='rate2', expression= 0.5)
            rate3 = gillespy2.Parameter(name='rate3', expression = 0.1)
            self.add_parameter([rate1,rate2,rate3])
            
            #Species
            A = gillespy2.Species(name='A', initial_value=301)
            B = gillespy2.Species(name='B', initial_value=120)
            C = gillespy2.Species(name='C', initial_value=0)
            D = gillespy2.Species(name='D', initial_value=0)
            self.add_species([A, B, C, D])
            
            #reactions
            r1 = gillespy2.Reaction(name="r1",reactants={A:1,B:1}, products={C:1},
                   rate=rate1)
            
            r2 = gillespy2.Reaction(name="r2",reactants={C:1}, products={A:1,B:1},
                    rate=rate2)
            
            r3 = gillespy2.Reaction(name="r3",reactants={C:1}, products={B:1,D:1},
                    rate=rate3)
            self.add_reaction([r1,r2,r3])
            self.timespan(numpy.linspace(0,100,101))

In [3]:
from gillespy2.solvers.cpp import SSACSolver
model = MichaelisMenten()
csolver = SSACSolver(model)

In [4]:
dresults = model.run(solver=csolver, show_labels=True)
print(dresults[0]['A'])

[301. 269. 244. 244. 242. 238. 235. 230. 211. 206. 206. 204. 201. 193.
 195. 187. 174. 175. 179. 170. 171. 159. 164. 168. 163. 160. 152. 145.
 135. 134. 139. 134. 134. 134. 125. 122. 120. 122. 115. 110. 111. 102.
 104. 100.  99.  93.  88.  81.  84.  85.  80.  78.  80.  71.  76.  72.
  75.  63.  68.  72.  70.  67.  66.  65.  61.  58.  54.  53.  53.  54.
  56.  56.  52.  47.  50.  49.  49.  42.  43.  44.  44.  44.  41.  39.
  37.  38.  36.  34.  36.  31.  33.  33.  37.  32.  30.  35.  27.  27.
  32.  29.  29.]


In [5]:
model = MichaelisMenten()
results = model.run(show_labels=False)
dresults = model.run(solver=csolver, show_labels=False)
plt.plot(dresults[0][:,0],dresults[0][:,1],'--r', label='Substrate')
plt.plot(dresults[0][:,0],dresults[0][:,4],'--b', label='Product')
plt.plot(results[0][:,0],results[0][:,1],'-r', label='Substrate')
plt.plot(results[0][:,0],results[0][:,4],'-b', label='Product')
plt.plot([0],[11])
plt.legend(loc='best')

InvalidModelError: Model must be either a GillesPy Model instance or an xml file name.

In [ ]:
import time
from gillespy2 import SSASolver, StochKitSolver, BasicSSASolver, SSACSolver
import numpy as np
log2_trajectories_max = 10
loops = 100
solvers = [StochKitSolver, csolver] #  SSASolver, BasicSSASolver
timing_data = np.zeros((log2_trajectories_max+1,len(solvers)+1))
for n in range(0,log2_trajectories_max+1):
    timing_data[n, 0] = 2**n
    for i in range(loops):
        for solver in range(len(solvers)):
            time_start = time.perf_counter()
            model.run(solver=solvers[solver], show_labels=False, number_of_trajectories=2**n)
            time_end = time.perf_counter()
            timing_data[n, solver+1] += (time_end - time_start)/loops
    print("{0}:\tStochKit:{1}\tSSACSolver:{2}\tRatio:{3}".format(timing_data[n,0], timing_data[n,1], timing_data[n, 2], (timing_data[n, 1]/timing_data[n, 2])))

In [ ]:
plt.plot(timing_data[:,0],timing_data[:,1],'-r', label='StochKit')
plt.plot(timing_data[:,0],timing_data[:,2],'-b', label='Cython')
plt.plot(timing_data[:,0],timing_data[:,3],'-g', label='Numpy')
plt.plot(timing_data[:,0],timing_data[:,4],'-k', label='Python')
plt.title("SSA Simulation Time")
plt.ylabel("Seconds")
plt.xlabel("Trajectories")
plt.legend(loc='best')
plt.savefig("SSA_Comparisons3.png")

2018-10-30 16:19:55,077 - tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x7f9f97a61bf8>)
Traceback (most recent call last):
  File "/home/dalton/environments/GillesPy2/venv/lib/python3.6/site-packages/tornado/ioloop.py", line 758, in _run_callback
    ret = callback()
  File "/home/dalton/environments/GillesPy2/venv/lib/python3.6/site-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/dalton/environments/GillesPy2/venv/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 306, in advance_eventloop
    eventloop(self)
TypeError: 'NoneType' object is not callable
